In [27]:
! pip install nltk, joblib

ERROR: Invalid requirement: 'nltk,'


# Imports

In [28]:
import pandas as pd
import nltk
import re
import chardet
import pandas as pd

from sklearn.utils import resample
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/k2257956/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/k2257956/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Globals

In [29]:
DATASET_PATH = './../datasets/SMS Spam Collection/'
KAGGLE_FILE_PATH = f'{DATASET_PATH}kaggle/spam.csv'
SPAMBASE_FILE_PATH = f'{DATASET_PATH}spambase/SMSSpamCollection'
EXPORT_FILE_PATH = f'./../datasets/SMS Spam Collection/final/spam.csv'

# Data Cleaning

## Kaggle

In [30]:
with open(KAGGLE_FILE_PATH, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
    
df_kaggle = pd.read_csv(KAGGLE_FILE_PATH, encoding=result['encoding'])

In [31]:
df_kaggle.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [32]:
df_kaggle = df_kaggle.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [33]:
df_kaggle = df_kaggle.rename(columns={'v1': 'target', 'v2': 'text'})

In [34]:
df_kaggle.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Spambase

In [35]:
# Read the data into a DataFrame
df_spambase = pd.read_csv(SPAMBASE_FILE_PATH, sep="\t", names=["target", "text"])
 

In [36]:
df_spambase.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Data Concatination

In [37]:
df_final = df_kaggle

In [38]:
df_final['target'] = df_final['target'].map({'ham': 0, 'spam': 1})

In [39]:
df_final.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


# Downsampling

In [40]:
# Count the instances of spam and non-spam.
spam = df_final[df_final['target'] == 1]
non_spam = df_final[df_final['target'] == 0]

print(f'Original non-spam count: {len(non_spam)}')
print(f'Original spam count: {len(spam)}')

# Undersample the non-spam emails: Randomly select "x" amount (equal to spam count) from non-spam data.
non_spam_downsampled = resample(non_spam, replace=False, n_samples=len(spam), random_state=42)

print(f'Undersampled non-spam count: {len(non_spam_downsampled)}')

# Concatenate the down-sampled non-spam with spam data to create a new balanced dataframe.
df_balanced = pd.concat([non_spam_downsampled, spam])

# Show the new counts.
print(df_balanced['target'].value_counts())

Original non-spam count: 4825
Original spam count: 747
Undersampled non-spam count: 747
1    747
0    747
Name: target, dtype: int64


# Data Export

In [41]:
df_balanced.to_csv(EXPORT_FILE_PATH, index=False)